In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import set_visible_GPUs
import tf_lab.point_clouds.in_out as pio
import tf_lab.models.point_net_based_AE as pnAE
from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id
from tf_lab.point_clouds.in_out import PointCloudDataSet
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf

from tf_lab.autopredictors.evaluate.basics import eval_model, read_saved_epochs
from tf_lab.autopredictors.plotting.basics import plot_original_pclouds_vs_reconstructed,\
                                                  plot_train_val_test_curves, plot_reconstructions_at_epoch


from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from geo_tool import Point_Cloud

/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [8]:
seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
full_pclouds_path = osp.join(top_data_dir, 'ShapeNetPointClouds/from_manifold_meshes/1024/')

experiment_name = 'enc_filter_1_complex_decoder_full_SN_1'
train_dir = osp.join(top_data_dir, 'OUT/models/vanilla_ae', experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/vanilla_ae/enc_filter_1_complex_decoder_full_SN_1'

In [4]:
full_file_names = pio.load_filenames_of_input_data(full_pclouds_path)
full_pclouds, full_model_names, class_ids = pio.load_crude_point_clouds(file_names=full_file_names, n_threads=11)
print '%d files containing complete point clouds were found.' % (len(full_pclouds), )

6778 files containing complete point clouds were found.


In [7]:
train_data_, val_data_, test_data_ = pio.train_validate_test_split([full_pclouds,                                                             
                                                            full_model_names],
                                                            train_perc=0.8,
                                                            validate_perc=0.1,
                                                            test_perc=0.1,
                                                            seed=seed)

train_data = PointCloudDataSet(train_data_[0], labels=train_data_[1])
val_data = PointCloudDataSet(val_data_[0], labels=val_data_[1])
test_data = PointCloudDataSet(test_data_[0], labels=test_data_[1])

In [9]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

# set_visible_GPUs([3])

conf = Conf(n_input = [1024, 3],
               training_epochs = 1000,
               batch_size = 80,
               loss = 'Chamfer',
               train_dir = train_dir,
               loss_display_step = 1,
               saver_step = 5,
               learning_rate = 0.00002,
               saver_max_to_keep = 200,
               gauss_augment = {'mu': 0,'sigma': 0.02},
               encoder = pnAE.encoder,
               decoder = pnAE.decoder
               )

reset_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

In [5]:
ae.restore_model(conf.train_dir, 99)

NotFoundError: Key enc_filter_1_complex_decoder_full_SN/BatchNormalization/is_training not found in checkpoint
	 [[Node: enc_filter_1_complex_decoder_full_SN_1/save/RestoreV2_2 = RestoreV2[dtypes=[DT_BOOL], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_enc_filter_1_complex_decoder_full_SN_1/save/Const_0, enc_filter_1_complex_decoder_full_SN_1/save/RestoreV2_2/tensor_names, enc_filter_1_complex_decoder_full_SN_1/save/RestoreV2_2/shape_and_slices)]]
	 [[Node: enc_filter_1_complex_decoder_full_SN_1/save/RestoreV2_45/_143 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_235_enc_filter_1_complex_decoder_full_SN_1/save/RestoreV2_45", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'enc_filter_1_complex_decoder_full_SN_1/save/RestoreV2_2', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_mod

TypeError: graph_def must be a GraphDef proto.